# Optimization techniques Lab. 4: (Reduced) Variable Neighborhood Search
## Introduction
**Goal.** The goal of this lab is to compare the behavior of VNS and RVNS on the knapsack 0/1 problem.

**Getting started.** The following cells contain the implementation of the methods that we will use throughout this lab, together with utilities.

## The Knapsack 0/1 problem
The knapsack 0/1 problem is a combinatorial problem that works as follows.

We have a bag with limited capacity L, and we have a set of items $I = {i_1, ..., i_N}$
Each $i_j$ has a given volume $w_{i_j}$ and a value $v_{i_j}$.
The goal of the optimization problem is to fill the bag with a combination of items $S$ such that the combination maximizes the total value contained by the bag while complaining with the constraint on the capacity.

In this version of the knapsack problem we can either discard or carry (at most 1) item. Thus, the search space is $\{0, 1\}^{|I|}$.

Since this is a maximization problem, we can turn it into a minimization problem by returning the opposite of the value.
If a solution is not feasible (i.e., exceeds the maximum volume) the function will return a score of 0 (like an empty bag).

In [ ]:
import numpy as np
import random
import itertools

from matplotlib import pyplot as plt
from typing import Callable, List, Tuple, NoReturn
from collections import Counter, deque

In [ ]:
class Knapsack_0_1:

    def __init__(self):
        self._items = [
            {'name': 'apricot',    'value': 1, 'volume': 1},    
            {'name': 'apple',      'value': 1, 'volume': 2},    
            
            {'name': 'cherry',     'value': 2, 'volume': 1},    
            {'name': 'pear',       'value': 2, 'volume': 2},    
            {'name': 'banana',     'value': 2, 'volume': 2},    
            
            {'name': 'blueberry',  'value': 3, 'volume': 1},    
            {'name': 'orange',     'value': 3, 'volume': 2},    
            {'name': 'avocado',    'value': 3, 'volume': 2},    
            
            {'name': 'coconut',    'value': 4, 'volume': 3},    
            
            {'name': 'watermelon', 'value': 5, 'volume': 10},    
        ]
        self._BAG_CAPACITY = 10
        self.history = []
        self.values = []

    def _get_value(self, solution):
        cur_cap = self._BAG_CAPACITY
        cur_val = 0
        for i, v in enumerate(solution):
            if v == 1:
                cur_val += self._items[i]['value']
                cur_cap -= self._items[i]['volume']
            if cur_cap < 0:
                return 0
        return -cur_val

    def __call__(self, solution):
        value = self._get_value(solution)
        self.history.append(solution)
        self.values.append(value)
        return value
    
    def trend(self):
        # 4 + 3 + 2 + 1 + 3 + 3 = 16
        # 3 + 1 + 1 + 1 + 2 + 2 = 10
        plt.figure()
        plt.plot(self.values)
        plt.axhline(-16, color="r", label="optimum")
        plt.show()

In [ ]:
def shake(
    x: List[bool], 
    k: int
) -> List[bool]:

    ss = Counter(random.choices(range(len(x)), k = 3 * k))
    return [ e if ss.get(i, 0) % 2 == 0 else not e for i, e in enumerate(x) ]


def move_or_not(
    f: Callable[[List[bool]], int], 
    x: List[bool], 
    xp: List[bool], 
    k: int
) -> Tuple[List[bool], int]:
    if f(xp) < f(x):
        x = xp
        k = 1
    else:
        k += 1
    return x, k


def local_search(
    fn: Callable[[List[bool]], int], 
    x: List[bool]
) -> List[bool]:

    ts = { i for (i, e) in enumerate(x) if e }
    fs = [ i for (i, e) in enumerate(x) if not e ]

    xps = [ ts ] + [ ts.union([f]) for f in fs ]
    
    return min([ [ i in xp for i in range(len(x)) ] for xp in xps ], key = fn)


def vns(
    f: Callable[[List[bool]], int],
    x0: List[bool],
    kmax: int
) -> None:
    """
    Seeks for the minimum of the function by means of the Variable Neighborhood
    Search algorithm.

    :f: the function to optimize
    :x0: the initial point
    :kmax: the max number of iterations for each neighborhood
    """
    x = [e for e in x0]
    k = 0
    while k < kmax:
        x, k = move_or_not(f, x, local_search(f, shake(x, k)), k)


def rvns(
    f: Callable[[List[bool]], int], 
    x0: List[bool], 
    kmax: int
) -> None:
    """
    Seeks for the minimum of the function by means of the Reduced Variable
    Neighborhood Search algorithm.

    :f: the function to optimize
    :x0: the initial point
    :kmax: the max number of iterations for each neighborhood
    """
    x = [e for e in x0]
    k = 0
    while k < kmax:
        x, k = move_or_not(f, x, shake(x, k), k)

# Variable Neighborhood Search
---
## Questions:
- how does the starting point influence the search process?
- how does the kmax parameter affect the quality of the result?
- how does the generation of the neighborhood affect:
    - quality of the search?
    - velocity of the search?

In [ ]:
x0s = [[False] * 10, [ i % 2 == 0 for i in range(10) ]]

for x0 in x0s + [ [ not e for e in x0 ] for x0 in x0s ]:
  for kmax in [3, 6, 9, 12, 15]:
    random.seed(7)
    func = Knapsack_0_1()
    vns(
        f = func, 
        x0 = x0, 
        kmax= kmax
    )
    print(f'x0 : {x0} kmax : {kmax}')
    func.trend()

# Reduced Variable Neighborhood Search
---
## Questions:
- how does the starting point influence the search process?
- how does the kmax parameter affect the quality of the result?
- how does the generation of the neighborhood affect:
    - quality of the search?
    - velocity of the search?
- how does RVNS compare to VNS?

In [ ]:
x0s = [[False] * 10, [ i % 2 == 0 for i in range(10) ]]

for x0 in x0s + [ [ not e for e in x0 ] for x0 in x0s ]:
  for kmax in [34, 35]:
    random.seed(5)
    func = Knapsack_0_1()
    rvns(
        f = func, 
        x0 = x0, 
        kmax = kmax
    )
    print(f'x0 : {x0} kmax : {kmax}')
    func.trend()